In [11]:
import os
import glob
import argparse
import pandas as pd
from tqdm import tqdm
from multiprocessing import Pool

##########################################
parser = argparse.ArgumentParser()
parser.add_argument("--dataDir", "-d", help="The run directory")
parser.add_argument("--prefix", "-p", help="The prefix of input fastq file")
parser.add_argument("--thread", "-t", help="Thread number to run")
args = parser.parse_args()
data_dir = args.dataDir
prefix = args.prefix
file_number = int(args.thread)
#########################
# data_dir = "/home/galaxy/project/m6AQTL/2019_10_10/transcriptome_snp/SNPiR/shell/GATK_best_practices/heart_3"
# prefix= "heart_3_L6-"
# file_number = 25
##########################################
filtered_result = "%s/%srmhex.rmsk.rmintron.rmhom.rmblat.rmedit.bed" % (data_dir, prefix)
raw_vcf = "%s/%sUnifiedGenotyper_variants.vcf" % (data_dir, prefix)
filtered_vcf = "%s/%sfiltered_variants.vcf" % (data_dir, prefix)

df = pd.read_table(filtered_result, sep="\t", header=None)
chr_list, pos_list = df.iloc[:, 0].tolist(), df.iloc[:, 2].tolist()
query_list = ["%s\t%s" % (chr_list[i], pos_list[i]) for i in range(len(chr_list))]
# ! grep "#" $raw_vcf > $filtered_vcf
os.system("grep '#' %s > %s" (raw_vcf, filtered_vcf))
df_vcf = pd.read_table(raw_vcf, sep="\t", header=None, comment="#")
vchr_list, vpos_list = df_vcf.iloc[:, 0].tolist(), df_vcf.iloc[:, 1].tolist()
search_list = ["%s\t%s" % (vchr_list[i], vpos_list[i]) for i in range(len(vchr_list))]
#
def index_value(x_term):
    return search_list.index(x_term)
line_nums = []
pool = Pool(processes=file_number)
for x in query_list:
    line_nums.append(pool.apply_async(index_value, (x, )))
pool.close()
pool.join()

line_nums = [int(x.get()) for x in line_nums]
df_filter = df_vcf.iloc[line_nums, :]
with open(filtered_vcf, 'a') as fw:
    df_filter.to_csv(fw, sep="\t", header=False, index=False)